# Importaciones

In [1]:
from imports import *                    # pandas, numpy, etc.
from trading_environment import (
    DataManager,
    Optimizer,
    PerformanceAnalyzer,
    ReportManager,
    RiskManager,
    DATA_DIR,
    MAX_FILE_SIZE,
    REPORT_DIR
)
from strategies import (
    MovingAverageCrossStrategy,
    DcaStrategy,
    DonchianBreakoutStrategy,
    ADXTrendStrategy,
    ROCStrategy,
    MACDStrategy,
    BollingerMeanRevStrategy,
    RSIStrategy,
    PairTradingStrategy,
    VWAPStrategy
)

d:\Desktop\Algorithmic Trading\Prueba\venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Strategies

In [2]:
# 1) Load data
dm = DataManager(data_dir=DATA_DIR, max_file_size=MAX_FILE_SIZE)
data = dm.load_data(
    symbols='BTC-USD',
    period='2y',
    interval='1d'
)

2025-06-27 15:13:15,777 - INFO - Cargados 732 registros de caché (1 archivos)


In [3]:
strategies = [
    MovingAverageCrossStrategy, DcaStrategy, DonchianBreakoutStrategy,
    ADXTrendStrategy, ROCStrategy, MACDStrategy, BollingerMeanRevStrategy,
    RSIStrategy, PairTradingStrategy, VWAPStrategy
]

# run Bayesian optimisation on each and pick the best overall by Sharpe:
best_strategy, best_params, summary_df = Optimizer.find_best_strategy(
    strategies,
    data,
    method='bayes',
    metric='Sharpe',
    n_trials=50,       # passed into optimize_bayesian
    seed=42
)



[I 2025-06-27 15:13:15,818] A new study created in memory with name: no-name-04f57f82-dccd-41bb-8708-c52453f91192
[I 2025-06-27 15:13:15,829] Trial 0 finished with value: 0.5996530870011627 and parameters: {'short_window': 40, 'long_window': 200}. Best is trial 0 with value: 0.5996530870011627.
[I 2025-06-27 15:13:15,837] Trial 1 finished with value: 0.60438381093577 and parameters: {'short_window': 75, 'long_window': 130}. Best is trial 1 with value: 0.60438381093577.
[I 2025-06-27 15:13:15,845] Trial 2 finished with value: 0.3839188267649449 and parameters: {'short_window': 20, 'long_window': 40}. Best is trial 1 with value: 0.60438381093577.
[I 2025-06-27 15:13:15,853] Trial 3 finished with value: 0.5404360526103191 and parameters: {'short_window': 10, 'long_window': 180}. Best is trial 1 with value: 0.60438381093577.
[I 2025-06-27 15:13:15,864] Trial 4 finished with value: 0.44745979818329934 and parameters: {'short_window': 65, 'long_window': 150}. Best is trial 1 with value: 0.60

In [4]:
print("Best strategy:", best_strategy)
print("Params:       ", best_params)

Best strategy: <strategies.DcaStrategy object at 0x0000026605316DD0>
Params:        {'amount': 601.9542320695084, 'freq': 'W'}


In [5]:
summary_df

,Total Return,Ann. Return,Ann. Vol,Sharpe,Max Drawdown,Win Rate,Profit Factor,Expectancy
strategy,,,,,,,,
DcaStrategy,2.836206,0.588603,0.394769,1.368815,-0.281445,0.510929,1.289113,0.002124
RSIStrategy,0.985263,0.266273,0.189311,1.339967,-0.150198,0.088798,1.959525,-0.014605
MovingAverageCrossStrategy,0.160253,0.052502,0.039728,1.307543,-0.001656,0.009563,49.514412,-0.001330
VWAPStrategy,2.469383,0.534576,0.402753,1.263555,-0.281445,0.527322,1.257557,0.001997
MACDStrategy,0.317968,0.099711,0.086352,1.142810,-0.051127,0.031421,3.009816,-0.013228
DonchianBreakoutStrategy,0.678097,0.195077,0.197518,0.999340,-0.150534,0.080601,1.600999,-0.014156
ADXTrendStrategy,0.050272,0.017029,0.021334,0.801978,0.000000,0.002732,NaN,0.000068
ROCStrategy,0.922999,0.252458,0.401025,0.761409,-0.418827,0.491803,1.150970,0.000352
BollingerMeanRevStrategy,0.015513,0.005314,0.009102,0.586739,0.000000,0.001366,NaN,0.000021


# FIN